In [34]:
import os, cv2, numpy as np, random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 1. 라벨 정의
categories = ['rok', 'dprk']
label_map = {name: idx for idx, name in enumerate(categories)}

# 2. 이미지 로딩
data = []
data_dir = '/content/datasets/'

for category in categories:
    path = os.path.join(data_dir, category)
    for fname in os.listdir(path):
        if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(path, fname)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (64, 64))
                data.append([img, label_map[category]])
random.shuffle(data)

# 3. X, y 분리
X, y = zip(*data)
X = np.array(X)
y = np.array(y)

# 4. 훈련/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# 5. 데이터 증강기
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,
    brightness_range=[0.8, 1.0],
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=[0.8, 1.2]
)
datagen.fit(X_train)

# 6. 모델 구성
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(64,64,3)))
model.add(MaxPooling2D(2))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 7. 모델 학습
model.fit(
    datagen.flow(X_train, y_train, batch_size=16),
    validation_data=(X_test / 255.0, y_test),
    epochs=30
)

# 8. 테스트 평가
def evaluate_test_set(test_dir):
    correct = 0
    total = 0

    label_map = {'test_rok': 0, 'test_dprk': 1}

    for label_folder in os.listdir(test_dir):
        folder_path = os.path.join(test_dir, label_folder)
        if not os.path.isdir(folder_path):
            continue
        for fname in os.listdir(folder_path):
            if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(folder_path, fname)
                img = cv2.imread(img_path)
                if img is None:
                    continue
                img_resized = cv2.resize(img, (64, 64)) / 255.0
                img_input = img_resized.reshape(-1, 64, 64, 3)

                pred = model.predict(img_input)[0][0]
                pred_class = 1 if pred > 0.5 else 0
                true_class = label_map[label_folder]

                if pred_class == true_class:
                    correct += 1
                total += 1

    acc = correct / total
    print(f"\n테스트 정확도 : {acc*100:.2f}%")

# 9. 테스트 평가 실행
evaluate_test_set('/content/test')

Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 436ms/step - accuracy: 0.4781 - loss: 0.7375 - val_accuracy: 0.2500 - val_loss: 0.7527
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step - accuracy: 0.6308 - loss: 0.6800 - val_accuracy: 0.2500 - val_loss: 0.7239
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step - accuracy: 0.4337 - loss: 0.6875 - val_accuracy: 0.2500 - val_loss: 0.7353
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms/step - accuracy: 0.5642 - loss: 0.6920 - val_accuracy: 0.2500 - val_loss: 0.7678
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step - accuracy: 0.5427 - loss: 0.6906 - val_accuracy: 0.2500 - val_loss: 0.7795
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 161ms/step - accuracy: 0.5427 - loss: 0.7010 - val_accuracy: 0.2500 - val_loss: 0.7570
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accuracy: 0.6315 - loss: 0.6631 - val_accuracy: 0.2500 - val_loss: 0.7740
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step - accuracy: 0.5954 - loss: 0.6444 - val_accuracy: 0.2500 - val_loss: